# Shortest Path Problem

Some of this content came from the great [QuantEcon lecture on shortest path problems](https://python.quantecon.org/short_path.html)!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Problem Description

The shortest path problem is one of finding how to traverse a [graph](https://en.wikipedia.org/wiki/Graph_%28mathematics%29) from one specified node to another at minimum cost.

Consider the following graph

<img src="https://s3-ap-southeast-2.amazonaws.com/python.quantecon.org/_static/lecture_specific/short_path/graph.png" style="">

  
We wish to travel from node (vertex) A to node G at minimum cost

- Arrows (edges) indicate the movements we can take.  
- Numbers on edges indicate the cost of traveling that edge.

(Graphs such as the one above are called weighted [directed graphs](https://en.wikipedia.org/wiki/Directed_graph).)


### Notation

First, we need some way to represent the actions that one can take at each node.

Notationally, we will use $v$ to represent node $v$ and $c(v, w)$ to represent the cost of traveling from $v$ to $w$.

For a given path $v \rightarrow w \rightarrow x$ the cost of that path would be given by $c(v, w) + c(w, x)$.

### Finding the solution

Stare at our baseline graph. How would you find the shortest path?

<img src="https://s3-ap-southeast-2.amazonaws.com/python.quantecon.org/_static/lecture_specific/short_path/graph.png" style="">

If you are like me, the most straightforward way to find the solution for this graph is to traverse each of the possible paths and determine which is the shortest one.

We will call this the "brute force solution"

Two equal distance solutions:

* $A \rightarrow C \rightarrow F \rightarrow G$ costs 8
* $A \rightarrow D \rightarrow F \rightarrow G$ costs 8

## Coding the brute force solution

We proceed to code up the brute force solution.

### Code to solve

We will represent the graph inside of a class:

In [ ]:
class BruteForceShortestPath(object):
    """
    Solves the shortest path problem using brute force
    to traverse all available paths and then storing
    any path that is equal to the minimum distance

    Example
    -------
    
    ```python
    cost_dict = {
        "A": {"B": 1, "C": 5},
        "B": {"D": 9, "E": 6},
        "C": {"F": 2},
        "D": {"F": 4, "G": 8},
        "E": {"G": 4},
        "F": {"G": 1}
    }
    bfsp = BruteForceShortestPath(cost_dict, "A", "G")
    
    sol = bfsp.find_shortest_path()
    ```
    """
    def __init__(self, dag: dict, vertex: str, terminal: str):
        self.dag = dag
        self.vertex = vertex
        self.terminal = terminal

    def is_at_end(self, v):
        "Boolean denoting whether at the terminal node"
        return (v == self.terminal)

    def available_actions(self, v):
        "Available actions at node v"
        if self.is_at_end(v):
            return []
        else:
            return list(self.dag[v].keys())    

    def evaluate_path(self, cpath):
        "Evaluates the cost of a given path"
        cost = sum(
            [
                self.dag[cpath[i]][cpath[i+1]]
                for i in range(len(cpath) - 1)
            ]
        )

        return cost

    def step_back_path(self, cpath):
        # Remove last element from the path
        cpath = cpath[:-1]
        cnode = cpath[-1]
        
        return cnode, cpath

    def traverse_paths(self):
        # Allocate dictionary to store costs of
        # each path
        paths_costs = {}

        # Allocate another dictionary to store
        # available paths to use for each node
        available_actions = {
            k: self.available_actions(k)
            for k in self.dag.keys()
        }

        cnode = self.vertex
        cpath = [cnode]
        while len(cpath) > 0:

            # Current set of actions -- This is
            # a pointer to the list inside the
            # dict so when we pop, it will remove
            # items from the dict's list
            cactions = available_actions[cnode]

            # If we are at a terminal point, evaluate
            # the cost of that path and save the output
            # into the `paths_costs` dict then remove
            # last element of the current path
            if self.is_at_end(cnode):
                # Evalute
                cost = self.evaluate_path(cpath)
                paths_costs[tuple(cpath)] = cost

                # Don't need to reset actions because we are
                # at the terminal node... Just step back
                cnode, cpath = self.step_back_path(cpath)
            elif (len(cactions) == 0) and (len(cpath) > 1):
                # If we are going to step back, we should reset
                # action space for that particular node
                available_actions[cnode] = self.available_actions(cnode)

                # Step back one level
                cnode, cpath = self.step_back_path(cpath)

            elif len(cactions) > 0:
                caction = cactions.pop()

                cnode = caction
                cpath.append(caction)
            else:
                break
        
        return paths_costs

    def find_shortest_paths(self):
        paths_costs = self.traverse_paths()

        min_cost = min(paths_costs.values())
        
        min_cost_paths = [
            _path
            for _path in paths_costs
            if paths_costs[_path] <= min_cost
        ]

        return min_cost_paths

Let's use the code we wrote to compute the solution and see whether it matches what we found!

In [ ]:
simple_cost_dict = {
    "A": {"B": 1, "C": 5, "D": 3},
    "B": {"D": 9, "E": 6},
    "C": {"F": 2},
    "D": {"F": 4, "G": 8},
    "E": {"G": 4},
    "F": {"G": 1},
    "G": {}
}

bfsp_simple = BruteForceShortestPath(simple_cost_dict, "A", "G")
simple_paths = bfsp_simple.find_shortest_paths()
print(simple_paths)

## Success!!!

The code successfully found the two shortest paths that traverse our graph.

Now imagine that due to the great success of our "innovative" algorithm that we received a contract to manage delivery paths for Delivery Inc, a large delivery company.

When an order is placed, Delivery Inc finds the closest warehouse with the product and gives us a DAG that describes all routes that the product could travel for delivery, conditional on where the couriers are.

Our "proprietary" algorithm is then responsible for finding the fastest delivery path.

## Graph formats

When an order is placed, the DAG that the conglomerate delivers is a text file with lines that take the following form:

```
node0, node1 0.04, node8 11.11, node14 72.21
```

Each line should be interpreted as:

* This component of the path begins at `node0`
* It can go to `node1`,  `node8` or `node14` and it takes `0.04` minutes, `11.11` minutes, or `72.21` minutes to get to those nodes, respectively.


In [ ]:
def generate_path_dict(filename):
    """
    Takes the proprietary delivery format and converts it
    into a dictionary that our algorithm can read
    """
    with open(filename, "r") as f:
        lines = f.readlines()

    cost_dict = {}
    for line in lines:
        node_name, routes_str = map(
            lambda x: x.strip(),
            line.split(",", maxsplit=1)
        )

        if len(routes_str) == 0:
            cost_dict[node_name] = {}
        else:
            possible_routes = routes_str.split(", ")
            route_dict = {}
            for route in possible_routes:
                _node, _cost = route.split(" ")
                route_dict[_node] = float(_cost)

            cost_dict[node_name] = route_dict

    return cost_dict

## The first order arrives


In [ ]:
del_inc_graph = generate_path_dict("graph.txt")

In [ ]:
%%time
bfsp = BruteForceShortestPath(del_inc_graph, "node0", "node99")

fastest_path_bf = bfsp.find_shortest_paths()
time_to_deliver_bf = bfsp.evaluate_path(fastest_path_bf[0])

In [ ]:
print(f"The fastest path is:\n {fastest_path_bf[0]}")
print(f"The delivery takes {time_to_deliver_bf:.2f} minutes to deliver")

# Dynamic Programming: An efficient solution

As time goes on, the delivery networks adds more and more possible routes and eventually makes is so that our algorithm takes longer to run than it actually takes to execute the delivery.

Delivery Inc says that they'll manage their own deliveries unless we're able to produce a more effective solution.

## An insight

As we stare at the two solutions to our simple example, we realize that they both pass through $F$ and once the path reaches $F$ the cost is always 1.

<img src="https://s3-ap-southeast-2.amazonaws.com/python.quantecon.org/_static/lecture_specific/short_path/graph4.png" style="">

<img src="https://s3-ap-southeast-2.amazonaws.com/python.quantecon.org/_static/lecture_specific/short_path/graph3.png" style="">

What if we had a function, $J(v)$, that could tell us the minimum number of minutes it would take to move from node $v$ to the end.

In our case, for example, we know that $J(F) = 1$ and $J(G) = 0$.

If this function existed, the shortest path would just be a sequence of choices, $w^*$, where:

$$w^* = \arg \min_{w \in A(v)} c(v, w) + J(w)$$

## How to find $J(v)$?

Let's begin by thinking about how $J(v)$ would be defined...

$$J(v) = \min_{w \in A(v)} c(v, w) + J(w)$$

This equation is known as the _Bellman equation_, after the mathematician Richard Bellman.

You should think about the Bellman equation as a restriction that $J$ must satisfy.

**Algorithm to find $J$**

We won't talk today about exactly why the algorithm that we propose works today, but we may revisit this topic soon.

1. Begin with some guess for $J(v)$ and denote it $J_0(v)$. One frequently used guess is $J_0(v) = 0$.
2. Create a new guess $J_{n+1}(v) = \min_{w \in A(v)} c(v, w) + J_n(w)$ for all $v$.
3. If $J_n(v) \neq J_{n+1}(v)$ return to step 2. Otherwise, $J_n(v) = J_{n+1}(v) = J(v)$.

We call this algorithm "value function iteration"

### Code for value function iteration

In [ ]:
class DynamicProgrammingShortestPath(object):
    """
    Solves the shortest path problem using dynamic
    programming to find the value function and
    corresponding optimal paths

    Example
    -------
    
    ```python
    cost_dict = {
        "A": {"B": 1, "C": 5},
        "B": {"D": 9, "E": 6},
        "C": {"F": 2},
        "D": {"F": 4, "G": 8},
        "E": {"G": 4},
        "F": {"G": 1}
    }
    dpsp = DynamicProgrammingShortestPath(cost_dict, "A", "G")
    
    sol = dpsp.find_shortest_path()
    ```
    """
    def __init__(self, dag: dict, vertex: str, terminal: str):
        self.dag = dag
        self.vertex = vertex
        self.terminal = terminal

    def is_at_end(self, v):
        "Boolean denoting whether at the terminal node"
        return True if v == self.terminal else False

    def available_actions(self, v):
        "Available actions at node v"
        if self.is_at_end(v):
            return []
        else:
            return list(self.dag[v].keys())    

    def evaluate_path(self, cpath):
        "Evaluates the cost of a given path"
        cost = sum(
            [
                self.dag[cpath[i]][cpath[i+1]]
                for i in range(len(cpath) - 1)
            ]
        )

        return cost

    def compare_J(self, Jn, Jnp1):
        return max([abs(Jn[k]-Jnp1[k]) for k in self.dag.keys()])

    def eval_action(self, v, w, J):
        return self.dag[v][w] + J[w]

    def bellman_equation(self, v, J):

        if v == self.terminal:
            return 0

        # Actions that can be taken at node v
        actions = self.available_actions(v)

        # Bellman equation one step
        J_np1 = {}
        for w in actions:
            J_np1[w] = self.eval_action(v, w, J)
        min_J_np1 = min(J_np1.values())

        return min_J_np1

    def find_cost_function(self):

        # All possible nodes
        nodes = self.dag.keys()

        # Initial guess
        J_n = {v: 100 for v in nodes}
        J_np1 = {v: 0 for v in nodes}

        while self.compare_J(J_n, J_np1) > 1e-10:
            # Copy from J_{n+1} to J_{n}
            J_n.update(J_np1)

            for node in nodes:
                J_np1[node] = self.bellman_equation(node, J_n)

        return J_np1

    def find_shortest_path(self, J):

        cnode = self.vertex
        cpath = [cnode]
        while not self.is_at_end(cnode):
            # Actions that can be taken at node v
            actions = self.available_actions(cnode)

            # Find optimal action according to J
            optimal_action = actions[0]
            min_cost = 1e10
            for w in actions:
                J_v = self.eval_action(cnode, w, J)
                if J_v < min_cost:
                    optimal_action = w
                    min_cost = J_v

            cnode = optimal_action
            cpath.append(cnode)
                
        return tuple(cpath)

### Verify on the simple graph

<img src="https://s3-ap-southeast-2.amazonaws.com/python.quantecon.org/_static/lecture_specific/short_path/graph4.png" style="">

In [ ]:
dpsp = DynamicProgrammingShortestPath(
    simple_cost_dict, "A", "G"
)

Jstar = dpsp.find_cost_function()
shortest_path_simple_dp = dpsp.find_shortest_path(Jstar)
cost_simple_dp = dpsp.evaluate_path(shortest_path_simple_dp)

print(shortest_path_simple_dp)
print(cost_simple_dp)

## How long to solve the Delivery Inc's problem?

Does our new algorithm perform better than the previous one?

As a benchmark, the brute force algorithm required between 5 and 6 seconds on my computer...

In [ ]:
%%time

dpsp = DynamicProgrammingShortestPath(
    del_inc_graph, "node0", "node99"
)

Jstar_dp = dpsp.find_cost_function()
fastest_path_dp = dpsp.find_shortest_path(Jstar_dp)
time_to_deliver_dp = dpsp.evaluate_path(fastest_path_dp)

The dynamic programing algorithm only required about 7 ms on my computer -- This is almost a 1000x speedup!

Below, we confirm that it arrived at the same solution.

In [ ]:
print(f"The fastest path is:\n {fastest_path_dp}")
print(f"The delivery takes {time_to_deliver_dp:.2f} minutes to deliver")

# Conclusion

Dynamic programming provides a recursive way to solve problems that initially seem like they should be solved sequentially.

Additionally, the innovations that have been made over the last 50+ years in dynamic programming have set the stage for reinforcement learning. A deep understanding of reinforcement learning begins with an understanding of dynamic programming.

## Some shortest path humor...

<img src="https://imgs.xkcd.com/comics/travelling_salesman_problem.png">

Attribution: [xkcd](https://xkcd.com/399/)